In [2]:
#| export
import pandas as pd
import foodcosts.core as core
import datetime

In [3]:
#| default_exp latem

In [4]:
core.get_shop_names()

['La Hulpe',
 'Fort Jaco',
 'Woluwe',
 'Tongres',
 'Wemmel',
 'CHATELAIN',
 'OVERIJSE',
 'LATEM']

In [38]:
id_chocolate_mouse_85gr = 51176
chocolate_mousses = [
    19772,
    id_chocolate_mouse_85gr
]


In [39]:
date_range = pd.date_range(
    datetime.datetime.strptime('2022-10-12', '%Y-%m-%d').date(),
    datetime.datetime.strptime('2022-10-13', '%Y-%m-%d').date()
)

In [40]:
date_range

DatetimeIndex(['2022-10-12', '2022-10-13'], dtype='datetime64[ns]', freq='D')

In [41]:
def missing_sales(start_date, end_date, df):
    date_range = pd.date_range(
        datetime.datetime.strptime(start_date, '%Y-%m-%d').date() if isinstance(start_date, str) else start_date,
        datetime.datetime.strptime(end_date, '%Y-%m-%d').date() if isinstance(end_date, str) else end_date
    )
    df_dates = list(set(df.index))
    missing_dates = [d for d in date_range if d.date() not in df_dates]
    return [str(d) for d in missing_dates]

In [46]:
def nbr_days_product_not_sold_this_month(product_id, shop_name):
    today = datetime.date.today()
    first_day = datetime.date(today.year, today.month, 1)
    
    chocolate_mouse = core.get_sales_by_day(
        first_day.strftime('%Y-%m-%d'), 
        today.strftime('%Y-%m-%d'), 
        [shop_name], 
        product_id
    )

    chocolate_mouse_df = pd.DataFrame(
        chocolate_mouse,
        columns=['shop','date','product','qty_sold','rev']
    )
    chocolate_mouse_df.fillna(value=0, inplace=True)
    chocolate_mouse_df.set_index('date', inplace=True)
    
    return missing_sales(
        first_day,
        today,
        chocolate_mouse_df.loc[ chocolate_mouse_df['shop'] == shop_name]
    )

LATEM = nbr_days_product_not_sold_this_month(
    product_id=id_chocolate_mouse_85gr,
    shop_name='LATEM'
)    

WOLUWE = nbr_days_product_not_sold_this_month(
    product_id=id_chocolate_mouse_85gr,
    shop_name='Woluwe'
)

print(len(LATEM))
print(len(WOLUWE))

8
8


In [15]:
def get_list_of_products_sold_for_given_date_and_shop(date, shop):
    pass

In [35]:
def get_products_sold_for_a_shop(start_date, end_date, shop_name):
    # Construct the SQL query
    query = f"""
        SELECT 
               v_shop.name AS shop_name, 
               DATE(v_pos_order.created_at) AS day, 
               v_product.id as product_id,
               v_product.name AS product_name, 
               SUM(v_pos_order_line.quantity) AS units_sold, 
               SUM(v_pos_order_line.quantity * v_product.price) AS total_sales
        FROM v_pos_order_line
        LEFT JOIN v_product ON v_product.id = v_pos_order_line.product_id
        LEFT JOIN v_pos_order ON v_pos_order.id = v_pos_order_line.order_id
        LEFT JOIN v_pos_session ON v_pos_session.id = v_pos_order.session_id
        LEFT JOIN v_pos ON v_pos.id = v_pos_session.pos_id
        LEFT JOIN v_shop ON v_shop.id = v_pos.shop_id
        WHERE v_pos_order.created_at BETWEEN '{start_date}' AND '{end_date}'
          AND v_shop.name IN ('{shop_name}')
        GROUP BY v_shop.id, v_shop.name, day, v_product.id, v_product.name
        ORDER BY v_shop.name, day ASC;
    """
    return pd.DataFrame(
        [row for row in core.make_sql_query(query)], 
        columns = ['shop_name','day','product_id','product_name','units_sold','total_sales']
    )

In [36]:
prods_sold = get_products_sold_for_a_shop(datetime.datetime.now() - datetime.timedelta(days=1), datetime.datetime.now(), 'Woluwe')

In [48]:
pd.set_option('display.max_rows', None)

In [49]:
# chocolate_prods = prods_sold[prods_sold['product'].str.contains(r'chocolate', case=False)]
display(
#     prods_sold.loc(prods_sold['product_name'])
    prods_sold[prods_sold['product_name'].str.contains(r'choc', case=False)]
)

,shop_name,day,product_id,product_name,units_sold,total_sales
35,Woluwe,2023-02-16,19723,TARTE MOELLEUX CHOCOLAT PORTION,1.000,4.55000
89,Woluwe,2023-02-16,49406,MELOCAKES ARTISANAL AU CHOCOLAT NOIR,1.000,7.3500000
97,Woluwe,2023-02-16,51176,MERVEILLEUX AU CHOCOLAT 110G,2.000,7.60000
